In [1]:
!pip install tensorflow #! used because pip is a command line tool, not a code command

In [3]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


In [0]:
%matplotlib inline

In [7]:
from google.colab import files
file = files.upload()
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.describe()

Saving test.csv to test (2).csv


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.00000,42000.000000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,...,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00300,0.011190,0.005143,0.000214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000381,0.001310,0.010548,0.027262,0.050905,0.066405,0.129571,...,3.772524,2.748905,1.796452,1.089905,0.563190,0.239571,0.093524,0.024833,0.000857,0.001405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006143,0.035833,0.082357,0.114905,0.178714,0.301452,0.413643,0.513667,0.558833,0.677857,0.60281,0.489238,0.340214,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.56812,1.626927,1.053972,0.043916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078072,0.232634,1.131661,2.310396,3.121847,3.259128,4.992894,...,26.957829,22.879248,18.595109,14.434439,10.517823,6.469315,3.976306,1.846016,0.139556,0.287891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.949803,2.350859,3.934280,4.543583,5.856772,7.219742,8.928286,10.004069,10.129595,11.254931,10.69603,9.480066,7.950251,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [44]:
# Prep data
X_train = train_data.iloc[:,1:].values  # Pixel values
Y_train = train_data.iloc[:,0].values # Image label (digit that's being represented by the pixel values)
X_test = test_data.values
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1) #reshaping training data in 28x28x1 squares, for all images in training dataset. Format: (nb_images, rows, cols, nb_color_channels(grayscale here))
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train.shape

(42000, 28, 28, 1)

In [45]:
# Check for null and missing values
train_data.isnull().any().describe()
test_data.isnull().any().describe()

count       784
unique        2
top       False
freq        462
dtype: object

In [0]:
# Normalize images to reduce illumination effect
X_train = X_train / 255.0
X_test = X_test / 255.0

In [47]:
Y_train = to_categorical(Y_train, num_classes=10) # one_hot_encoding of labels
# initialize a seed for reproducibility
rand_seed = 4
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.15, random_state=rand_seed)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(35700, 28, 28, 1) (35700, 10) (6300, 28, 28, 1) (6300, 10)


In [0]:
# Build CNN model
model = Sequential()
model.add(Conv2D(filters = 20, kernel_size = (7,7),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 20, kernel_size = (7,7),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 50, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 50, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(10, activation = "softmax"))


In [0]:
model.compile(optimizer=SGD(learning_rate=0.01,momentum=0.0),loss="categorical_crossentropy", metrics=["accuracy"])
epochs = 20
batch_size = 70

In [0]:
# Augment Data to expand training set
data_aug = ImageDataGenerator(rotation_range=10,zoom_range=0.2, width_shift_range=0.2)
data_aug.fit(X_train)

In [51]:
# Fit the model
output = model.fit_generator(data_aug.flow(X_train, Y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, Y_val), steps_per_epoch=X_train.shape[0] // batch_size)

Epoch 1/20
510/510 [==============================] - 264s 517ms/step - loss: 1.5815 - accuracy: 0.4633 - val_loss: 0.3979 - val_accuracy: 0.8971
Epoch 2/20
510/510 [==============================] - 261s 511ms/step - loss: 0.3653 - accuracy: 0.8914 - val_loss: 0.1953 - val_accuracy: 0.9468
Epoch 3/20
510/510 [==============================] - 260s 510ms/step - loss: 0.2173 - accuracy: 0.9345 - val_loss: 0.1306 - val_accuracy: 0.9625
Epoch 4/20
510/510 [==============================] - 256s 501ms/step - loss: 0.1676 - accuracy: 0.9503 - val_loss: 0.1138 - val_accuracy: 0.9621
Epoch 5/20
510/510 [==============================] - 256s 501ms/step - loss: 0.1437 - accuracy: 0.9565 - val_loss: 0.1043 - val_accuracy: 0.9686
Epoch 6/20
510/510 [==============================] - 265s 519ms/step - loss: 0.1235 - accuracy: 0.9622 - val_loss: 0.0875 - val_accuracy: 0.9732
Epoch 7/20
510/510 [==============================] - 267s 523ms/step - loss: 0.1108 - accuracy: 0.9656 - val_loss: 0.0862 -

In [55]:
# Checking validation accuracy
from sklearn import metrics
Y_pred = model.predict(X_val)
print('R^2 score:',metrics.r2_score(Y_val, Y_pred))
Y_pred_class = np.argmax(Y_pred, axis=1)   # Converting predictions to one hot vectors
Y_val_class = np.argmax(Y_val, axis=1)
print('Accuracy:',metrics.accuracy_score(Y_val_class, Y_pred_class)) # accuracy needs a vector of labels

R^2 score: 0.9686180128435817
Accuracy: 0.9809523809523809


In [0]:
# Predict test dataset
test_output = model.predict(X_test)
test_output = np.argmax(test_output, axis=1)
test_output = pd.Series(test_output, name="Label")
results = pd.concat([pd.Series(range(1,28001),name = "ImageId"),test_output],axis = 1)
#results.to_csv("mnist_testoutput.csv",index=False)

In [69]:
# Save file to drive
from google.colab import drive
drive.mount('drive')
results.to_csv('mnist_testoutput.csv', index=False)
!cp mnist_testoutput.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [70]:
# Sanity check
df = pd.read_csv('mnist_testoutput.csv')
df.shape

(28000, 2)